In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
FeatureSetB = pd.read_csv('MIMIC_Windows/FSB_1Ws.csv')
FeatureMortality = pd.read_csv('MIMIC_Windows/FSB_Mortality.csv')

In [3]:
numberOfWindows = 1
numberOfFeatures = 20

In [4]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)
Subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(Subject_id)

In [5]:
X = FeatureMortality['subject_id'].equals(Subject_id)

In [6]:
FeatureSet = FeatureSetB.drop(['subject_id'], axis=1)

In [7]:
FeatureSet = FeatureSet.to_numpy()

In [8]:
Labels = pd.DataFrame()
Labels['Expired'] = FeatureMortality['Expired'] 
y_values = Labels.to_numpy()
y_values = y_values.flatten()

In [9]:
nClients = NumSubjects
X = FeatureSet 

Testing CNN

In [10]:
import keras
from keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn import metrics
from keras.layers import Dense, Activation, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import tensorflow as tf

In [11]:
PR = [
      keras.metrics.AUC(name='prc',multi_label=True,curve='PR'),
    ]

In [12]:
np.random.seed(1)
tf.random.set_seed(1234)
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(numberOfWindows,numberOfFeatures)))#,padding = 'same'))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = keras.optimizers.RMSprop(learning_rate=0.001) 
model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=PR)
model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1, 64)             1344      
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 64)             4160      
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 100) 

2022-09-02 08:41:37.048582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-02 08:41:37.048740: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
import numpy as np
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn import preprocessing

acc_score = []
re_score = []
pre_score = []
f_score = []
auroc = []
auprc = []


In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
resample = NeighbourhoodCleaningRule() 
y = y_values
k = 5
ep = 75

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

X_test =  scaler.transform(X) 
Train_shape = X.shape

X = X.reshape(int(Train_shape[0]/numberOfWindows),numberOfWindows,numberOfFeatures)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
# enumerate the splits and summarize the distributions

for train_ix, val_ix in skf.split(X,y):
    # select rows
    X_train, X_val = X[train_ix], X[val_ix]
    y_train, y_val = y[train_ix], y[val_ix]
    
    resample.fit_resample(X_train[:,:,0], y_train)
    X_train_2 = X_train[resample.sample_indices_]
    y_train_2 = y_train[resample.sample_indices_]

    history4 = model.fit(X_train,y_train, epochs = ep,batch_size=64)
    y_pred = model.predict(X_val)
    y_pred = list(map(lambda x: 0 if x<0.5 else 1, y_pred))
    #class_labels = np.argmax(y_pred, axis=1)
    y_actu = pd.Series(y_val)
    y_pred = pd.Series(y_pred)
     
    sensitivity = recall_score(y_actu, y_pred,pos_label = 1, average='binary')
    precision = precision_score(y_actu, y_pred,pos_label = 1, average='binary')
    f1_value = f1_score(y_actu, y_pred,pos_label = 1, average='binary')
    prc = average_precision_score(y_actu, y_pred,pos_label = 1)
    roc = roc_auc_score(y_actu, y_pred)
    accuracy = accuracy_score(y_actu, y_pred)
    
    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)
    auroc.append(roc)
    auprc.append(prc)
   
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_roc_score = sum(auroc)/k
avg_prc_score = sum(auprc)/k

sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score
auroc = avg_roc_score
auprc = avg_prc_score


Epoch 1/100


2022-09-02 08:41:37.580653: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-02 08:41:38.047418: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - 2s 16ms/step - loss: 0.3283 - prc: 0.1634
Epoch 2/100
72/72 [==============================] - 1s 17ms/step - loss: 0.2299 - prc: 0.4441
Epoch 3/100
72/72 [==============================] - 1s 15ms/step - loss: 0.2134 - prc: 0.4948
Epoch 4/100
72/72 [==============================] - 1s 14ms/step - loss: 0.2016 - prc: 0.5316
Epoch 5/100
72/72 [==============================] - 1s 13ms/step - loss: 0.1947 - prc: 0.5529
Epoch 6/100
72/72 [==============================] - 1s 12ms/step - loss: 0.1900 - prc: 0.5760
Epoch 7/100
72/72 [==============================] - 1s 13ms/step - loss: 0.1796 - prc: 0.6228
Epoch 8/100
72/72 [==============================] - 1s 14ms/step - loss: 0.1765 - prc: 0.6259
Epoch 9/100
72/72 [==============================] - 1s 13ms/step - loss: 0.1692 - prc: 0.6596
Epoch 10/100
72/72 [==============================] - 1s 14ms/step - loss: 0.1682 - prc: 0.6533
Epoch 11/100
72/72 [==============================] - 1s 14ms

2022-09-02 08:43:15.890091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/100
72/72 [==============================] - 1s 13ms/step - loss: 0.1959 - prc: 0.7710
Epoch 2/100
72/72 [==============================] - 1s 13ms/step - loss: 0.1145 - prc: 0.8318
Epoch 3/100
72/72 [==============================] - 1s 12ms/step - loss: 0.0902 - prc: 0.8883
Epoch 4/100
72/72 [==============================] - 1s 12ms/step - loss: 0.0761 - prc: 0.9179
Epoch 5/100
72/72 [==============================] - 1s 13ms/step - loss: 0.0715 - prc: 0.9234
Epoch 6/100
72/72 [==============================] - 1s 12ms/step - loss: 0.0714 - prc: 0.9209
Epoch 7/100
72/72 [==============================] - 1s 13ms/step - loss: 0.0635 - prc: 0.9374
Epoch 8/100
72/72 [==============================] - 1s 12ms/step - loss: 0.0588 - prc: 0.9440
Epoch 9/100
72/72 [==============================] - 1s 12ms/step - loss: 0.0531 - prc: 0.9569
Epoch 10/100
72/72 [==============================] - 1s 13ms/step - loss: 0.0487 - prc: 0.9648
Epoch 11/100
72/72 [=============================

In [16]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
print('Recall of each fold - {}'.format(re_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Precision of each fold - {}'.format(pre_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('F1_Score of each fold - {}'.format(f_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('AUROC of each fold - {}'.format(auroc))
print('Avg AUROC : {}'.format(avg_roc_score))
print('AUPRC of each fold - {}'.format(auprc))
print('Avg AUPRC : {}'.format(avg_prc_score))  

accuracy of each fold - [0.8924825174825175, 0.9388111888111889, 0.9676573426573427, 0.9816433566433567, 0.9798775153105862]
Avg accuracy : 0.9520943841809985
Recall of each fold - [0.2912621359223301, 0.5533980582524272, 0.75, 0.8269230769230769, 0.8252427184466019]
Avg Reccall : 0.6493651979088872
Precision of each fold - [0.375, 0.7037037037037037, 0.8764044943820225, 0.9662921348314607, 0.9444444444444444]
Avg Precision : 0.7731689554723262
F1_Score of each fold - [0.32786885245901637, 0.6195652173913044, 0.8082901554404146, 0.8911917098445596, 0.8808290155440415]
Avg F1_score : 0.7055489901358672
AUROC of each fold - 0.8157471264058588
Avg AUROC : 0.8157471264058588
AUPRC of each fold - 0.5785261300094333
Avg AUPRC : 0.5785261300094333


In [17]:
import csv
import os.path
from datetime import datetime

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','AUROC','AUPRC','NumberOfWindows','Epochs','AUROC','AUPRC']
dict_data = [{'model-type':'CNN', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'NumberOfWindows':numberOfWindows,"Epochs":ep,'AUROC':auroc,'AUPRC':auprc}]
metric_file = "Results/Results_CNN.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")